Calling the Libraries

In [1]:
# Data processing
import pandas as pd
import numpy as np

# Train test split
from sklearn.model_selection import train_test_split

# Modeling
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Import accuracy_score to check performance
from sklearn.metrics import accuracy_score

In [2]:
labeled_data = pd.read_csv('cleaned_data.csv')


Import the data
and get an overview of the data.

In [3]:
print(labeled_data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0    As a woman you shouldn't complain about clea...  
1    boy dats cold...tyga dwn bad for cuffin dat ...  
2   Dawg  You ever fuck a bitch and she start to ...  
3             GAnderson based she look like a tranny  
4    The shit you hear about me might be true or ...  


Label Data

In [4]:
labeled_data=pd.read_csv("labeled_data.csv",names=["hate_speech","offensive_language","neither"])


In [5]:
labeled_data

,,,,hate_speech,offensive_language,neither
NaN,count,hate_speech,offensive_language,neither,class,tweet
0.0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1.0,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2.0,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3.0,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
...,...,...,...,...,...,...
25291.0,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
25292.0,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
25294.0,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
25295.0,6,0,6,0,1,youu got wild bitches tellin you lies


In [6]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24784 entries, (nan, 'count', 'hate_speech', 'offensive_language') to (25296.0, '3', '0', '0')
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   hate_speech         24784 non-null  object
 1   offensive_language  24784 non-null  object
 2   neither             24784 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [7]:
labeled_data["hate_speech"].value_counts()

hate_speech
0          18892
3           2790
1           1694
2           1200
6            103
5             54
4             35
9              5
8              5
7              5
neither        1
Name: count, dtype: int64

Train and test split 


In [8]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(labeled_data["hate_speech"], labeled_data["offensive_language"], test_size=0.2, random_state=42)


we will tokenize the review text using a tokenizer.


The labels for the reviews are converted to one-dimensional numpy arrays.

The tokenized ids are stored in a dictionary with the key `input_ids` and the labels are stored in a numpy array with the key `labels`.


In [9]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#should be the same name from hugging face library

# Tokenize the reviews
tokenized_data_train = tokenizer(X_train.to_list(), return_tensors="np", padding=True)
tokenized_data_test = tokenizer(X_test.to_list(), return_tensors="np", padding=True)

# Labels are one-dimensional numpy or tensorflow array of integers
labels_train = np.array(y_train)
labels_test = np.array(y_test)

# Tokenized ids
print(tokenized_data_train["input_ids"][0])

[101 121 102]


In [10]:
# COMPILE AND FIT

we will load the pretrained model from the Hugging Face library.

In [11]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


we will compile and fit the model.


In [12]:
# Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
model.compile(optimizer=Adam(5e-6), loss=loss, metrics=['accuracy'])

In [13]:
# Fit the model
model.fit(dict(tokenized_data_train),
          labels_train,
          validation_data=(dict(tokenized_data_test), labels_test),
          batch_size=4,
          epochs=2)

Epoch 1/2


2023-12-24 22:08:29.618679: W tensorflow/core/framework/op_kernel.cc:1816] OP_REQUIRES failed at cast_op.cc:122 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node Cast_1 defined at (most recent call last):
  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/Users/davidrothschild/Library/r-miniconda-arm64/lib/python3.9/asyncio/base_events.py", line 596, in run_forever

  File "

All the weights will be updated by default for the transfer learning model.



We run the predictions on the test data and get the predicted probabilities and labels.

In [14]:
# Predictions
y_test_predict = model.predict(dict(tokenized_data_test))['logits']

# First 5 predictions
y_test_predict[:5]

155/155 [==============================] - 11s 66ms/step


array([[0.36553594, 0.30390155],
       [0.41226524, 0.30849582],
       [0.41226524, 0.30849582],
       [0.41226524, 0.30849582],
       [0.41226524, 0.30849582]], dtype=float32)

In [15]:
# Predicted probabilities
y_test_probabilities = tf.nn.softmax(y_test_predict)

# First 5 predicted probabilities
y_test_probabilities[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[0.51540375, 0.48459628],
       [0.5259191 , 0.47408086],
       [0.5259191 , 0.47408086],
       [0.5259191 , 0.47408086],
       [0.5259191 , 0.47408086]], dtype=float32)>

In [16]:
# Predicted label
y_test_class_preds = np.argmax(y_test_probabilities, axis=1)

# First 5 predicted labels
y_test_class_preds

array([0, 0, 0, ..., 0, 1, 0])

We can check the accuracy of the model by comparing the predicted labels with the true labels.

In [18]:
accuracy_score(y_test_class_preds, y_test)

0.0

In [18]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()

# Fit and transform the true labels
y_test_encoded = le.fit_transform(y_test)

# Transform the predictions (do not fit again!)
y_test_class_preds_encoded = le.transform(y_test_class_preds)
rint("Confusion Matrix:\n", conf_matrix)


ValueError: y contains previously unseen labels: 1

We save the model and the tokenizer, so that we can use it later.


`tokenizer.save_pretrained` saves the tokenizer information to the drive and `model.save_pretrained` saves the model to the drive.

In [20]:
# Save tokenizer
tokenizer.save_pretrained('./sentiment_transfer_learning_tensorflow/')

# Save model
model.save_pretrained('./sentiment_transfer_learning_tensorflow/')

We can zip the folder and download it to our local drive.

In [31]:
!zip -r sentiment_transfer_learning_tensorflow.zip sentiment_transfer_learning_tensorflow/

updating: sentiment_transfer_learning_tensorflow/ (stored 0%)
updating: sentiment_transfer_learning_tensorflow/tokenizer_config.json (deflated 76%)
updating: sentiment_transfer_learning_tensorflow/special_tokens_map.json (deflated 42%)
updating: sentiment_transfer_learning_tensorflow/config.json (deflated 47%)
updating: sentiment_transfer_learning_tensorflow/tokenizer.json (deflated 70%)
updating: sentiment_transfer_learning_tensorflow/vocab.txt (deflated 49%)
updating: sentiment_transfer_learning_tensorflow/tf_model.h5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)


if you are using google colab, you can easily connect and copy your zip file (model) to any directory of your google drive